In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from dataclasses import dataclass

import matplotlib.pyplot as plt # visualization
plt.rcParams.update({'font.size': 14})
import seaborn as sns # visualization

import warnings # Supress warnings 
from tqdm import tqdm
from pathlib import Path
import json
import plotly.graph_objs as go
from PIL import Image
from datetime import datetime

from ahrs.filters import EKF
from ahrs.common.orientation import acc2q
from ahrs.utils import WMM
from scipy.spatial.transform import Rotation as R
from scipy import integrate


#AI
import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')

data=pd.read_csv('datasets\spiralStairs_CalInertialAndMag.csv')
time_ = data['Packet number']



from mpl_toolkits.mplot3d import Axes3D
def plot_3d_graph(x,y,z, c=None):
    sns.set(style = "darkgrid")

    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111, projection = '3d')

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    ax.scatter(x, y, z, c=c)
    ax.set_title("acce")
    plt.show()
def Concat(data,Pos_xyz):
    alldata=pd.concat([data,Pos_xyz],axis=1)
    alldata.dropna(inplace=True)
    return alldata

class Effecasity_data :
    
            def splitdata(self,alldata):
                 # Extract features from the raw data
                    features = alldata[['Gyroscope Y (deg/s)','Gyroscope Z (deg/s)','Accelerometer X (g)','Accelerometer Y (g)','Accelerometer Z (g)','Magnetometer X (G)','Magnetometer Y (G)','Magnetometer Z (G)']].values
                    target_pos = alldata[['pos_x', 'pos_y', 'pos_z']].values

                  
        
class Kalman:
    
            plot_3d_graph(
            np.cumsum(data['Accelerometer X (g)']),
            np.cumsum(data['Accelerometer Y (g)']),
            np.cumsum(data['Accelerometer Z (g)']),
            )

            def simple_integral(self):
                    print('integral simple')
                    vel_x = integrate.cumtrapz(data['Accelerometer X (g)'], time_, initial=0)
                    vel_y = integrate.cumtrapz(data['Accelerometer Y (g)'], time_, initial=0)
                    vel_z = integrate.cumtrapz(data['Accelerometer Z (g)'], time_, initial=0)

                    pos_x = integrate.cumtrapz(vel_x, time_, initial=0)
                    pos_y = integrate.cumtrapz(vel_y, time_, initial=0)
                    pos_z = integrate.cumtrapz(vel_z, time_, initial=0)
                    return pos_x,pos_y,pos_z
                
# ***********************************************************************************
            def integral_kalman(self):
                
                    print('integral with kalman filter')
                    ekf_estimator = EKF(gyr=data.iloc[:,1:4].values, acc=data.iloc[:,4:7].values, mag=data.iloc[:,7:12].values, frequency=100.0)
                    # Initializing the class with the sensor data generates our Quaternion in the Q Variable

                    Q = ekf_estimator.Q
                    n_samples = data.shape[0]
                    new_acce = np.zeros((n_samples, 3))
                      # Initializing Array to hold the Linear acceleration Vector
                    data_acc=data.iloc[:,4:7].values
                    for t in range(n_samples):
                        r = R.from_quat(Q[t])
                        # Getting a Rotation Matrix from the Quaternions
                        new_acce[t] = np.matmul(r.as_matrix().T,data_acc[t])
                    # matmul Rotation Matrix Transpose to orignal Acceleration to produce the clean linear acceleration
                    vel_x = integrate.cumtrapz(new_acce[:,0], time_, initial=0)
                    vel_y = integrate.cumtrapz(new_acce[:,1], time_, initial=0)
                    vel_z = integrate.cumtrapz(new_acce[:,2], time_, initial=0)

                    pos_x = integrate.cumtrapz(vel_x, time_, initial=0)
                    pos_y = integrate.cumtrapz(vel_y, time_, initial=0)
                    pos_z = integrate.cumtrapz(vel_z, time_, initial=0)
                    return pos_x,pos_y,pos_z
#**************************************************************************************
            def Dataframe(self,pos_x,pos_y,pos_z):

                    pos_x=pd.DataFrame(pos_x,columns=['pos_x'])
                    pos_y=pd.DataFrame(pos_y,columns=['pos_y'])
                    pos_z=pd.DataFrame(pos_z,columns=['pos_z'])
                    Pos_xyz=pd.concat([pos_x,pos_y,pos_z],axis=1)
                    return Pos_xyz


# ***********************************************************************************
if __name__ == '__main__':
    
    Kalman=Kalman()
    pos_x_s,pos_y_s,pos_z_s=Kalman.simple_integral()
    print('Simple integral')
    plot_3d_graph(pos_x_s,pos_y_s,pos_z_s)

    pos_x,pos_y,pos_z=Kalman.integral_kalman()
    Pos_xyz=Kalman.Dataframe(pos_x=pos_x,pos_y=pos_y,pos_z=pos_z)
    print(' integral + kalman ')
    plot_3d_graph(pos_x,pos_y,pos_z)
    
